In [1]:
# !pip install datasets transformers==4.18.0 sentencepiece

In [1]:
from datasets import load_dataset, load_from_disk
from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast, TrainingArguments, DataCollatorForLanguageModeling, Trainer, pipeline
from tokenizers import BertWordPieceTokenizer
import os
import json
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


# Load raw data and make text data

In [2]:
ROOT = ''
dataset = load_dataset("csv", data_files=ROOT, delimiter=',', column_names=['text'], split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# split the dataset into training (90%) and testing (10%)
d = dataset.train_test_split(test_size=0.1)
d["train"], d["test"]

(Dataset({
     features: ['text'],
     num_rows: 7792820
 }),
 Dataset({
     features: ['text'],
     num_rows: 865869
 }))

In [7]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

In [8]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True

In [9]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [10]:
epochs = 40
model_path = f""

# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [11]:
# save the tokenizer  
tokenizer.save_model(model_path)

['10000issues_5000stars_large_40epochs/vocab.txt']

In [12]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [14]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(model_path, padding=True)

In [15]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  # remove other columns, and remain them as Python lists
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

Map:   0%|          | 0/7792820 [00:00<?, ? examples/s]

Map:   0%|          | 0/865869 [00:00<?, ? examples/s]

In [16]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

In [17]:
train_dataset.save_to_disk('data/train')
test_dataset.save_to_disk('data/test')

Saving the dataset (0/74 shards):   0%|          | 0/7792820 [00:00<?, ? examples/s]

Saving the dataset (0/9 shards):   0%|          | 0/865869 [00:00<?, ? examples/s]

In [ ]:
# train_dataset = load_from_disk('data/train')
# test_dataset = load_from_disk('data/test')

In [18]:
len(train_dataset), len(test_dataset)

(7792820, 865869)

# Initialize model and trainer

In [19]:
# initialize the model with the config for BERT large

model_config = BertConfig(
    vocab_size=vocab_size,
    max_position_embeddings=max_length,
    hidden_size=1024,
    num_hidden_layers=24,
    num_attention_heads=16,
    intermediate_size=4096,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
)
model = BertForMaskedLM(config=model_config)

In [21]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM, Masked Language Model) task
# 텍스트의 각 배치(batch)에서 일부 토큰을 무작위로 마스킹
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [22]:
training_args = TrainingArguments(
    learning_rate=1e-5,
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,
    num_train_epochs=epochs,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=16, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=4,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=16,  # evaluation batch size
    logging_steps=200,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=50000,
    fp16=True,                      # memory save
    weight_decay=0.01,
    warmup_steps=2970,
    #dataloader_num_workers=2,
#     remove_unused_columns=False
    #optim="adafactor",
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
#     report_to=False               # when you don't want to log with wandb
)

#training_args = training_args.set_dataloader(train_batch_size=64, eval_batch_size=64)

In [23]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using amp fp16 backend


In [ ]:
# train the model
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running training *****
  Num examples = 7792820
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 4
  Total optimization steps = 1217600
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

········


wandb: Appending key for api.wandb.ai to your netrc file: /home/work/.netrc


Step,Training Loss,Validation Loss
200,9.354600,8.332954
400,8.179100,7.781520
600,7.612100,7.242273
800,7.078200,6.743359
1000,6.585100,6.324765
1200,6.204700,6.035371
1400,5.989700,5.887711
1600,5.869900,5.794020
1800,5.800600,5.734512
2000,5.748800,5.690677


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a correspond

  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text.
***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64
The following columns in the evaluation set  don't

***** Running Evaluation *****
  Num examples = 865869
  Batch size = 64


In [ ]:
model.save_pretrained(os.path.join(model_path, "checkpoint-last"))

# Masking test

In [ ]:
# when you load from pretrained
#model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-6000"))
#tokenizer = BertTokenizerFast.from_pretrained(model_path)
# or simply use pipeline

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [ ]:
# perform predictions
# https://huggingface.co/docs/transformers/main_classes/pipelines
#example = "It is known that [MASK] is the capital of Germany"
# jsonl
example = """
I have 3 input datasets and 2 output sets. My input: 1.[MASK] file, 2.yaml file, 3. gzip library as a dataset.
"""
for prediction in fill_mask(example):
  print(prediction)

In [8]:
# perform predictions
examples = [
'''Hello!

I ran into a problem in trying to implement my own [MASK] inherited from UserString and str. Let me demonstrate a piece of code:

from collections import UserString''',
'''class MyString(UserString, str):
    def  __init__([MASK], sec):
        self.sec = sec

    def __new__(cls, *args, **kwargs):
        return str.__new__(cls)

    @property
    def data(self):
        return self.sec''',
'''
assert MyString('lol kek') not in 'kek'
The result:

Traceback (most recent call last):
  File "/Users/pomponchik/Desktop/Projects/lazy_fstrings/test.py", line 16, in <module>
    assert MyString('lol kek') not in 'kek'
[MASK] Error
In this situation, the check does not pass. The string "kek" is sure to include "lol kek", although this is obviously impossible since "lol kek" is longer than just "kek".
'''
]
ans = ['class', 'self, sec', 'self']
for i, example in enumerate(examples):
  print('target:', ans[i])
  for prediction in fill_mask(example):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")
  print("="*50)


target: class
hello! i ran into a problem in trying to implement my own class inherited from userstring and str. let me demonstrate a piece of code : from collections import userstring, confidence: 0.2269645482301712
hello! i ran into a problem in trying to implement my own collections inherited from userstring and str. let me demonstrate a piece of code : from collections import userstring, confidence: 0.12955789268016815
hello! i ran into a problem in trying to implement my own classes inherited from userstring and str. let me demonstrate a piece of code : from collections import userstring, confidence: 0.08996809273958206
hello! i ran into a problem in trying to implement my own string inherited from userstring and str. let me demonstrate a piece of code : from collections import userstring, confidence: 0.06263256072998047
hello! i ran into a problem in trying to implement my own types inherited from userstring and str. let me demonstrate a piece of code : from collections import us

In [13]:
# perform predictions

examples = [

"The [MASK] is thrown when an application attempts to use null in a case where an object is required.",

"[MASK] is a proprietary issue tracking product developed by Atlassian that allows bug tracking and agile project management",

"[MASK] is a software tool for automating software build processes.",

"In object-oriented programming, a [MASK] is an extensible program-codetemplate for creating objects.",

"I have to discuss this with the other [MASK]."
]

for i, example in enumerate(examples):
  for prediction in fill_mask(example):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")
  print("="*50)


the exception is thrown when an application attempts to use null in a case where an object is required., confidence: 0.7306296229362488
the error is thrown when an application attempts to use null in a case where an object is required., confidence: 0.08684872835874557
the nullpointerexception is thrown when an application attempts to use null in a case where an object is required., confidence: 0.03762815520167351
the npe is thrown when an application attempts to use null in a case where an object is required., confidence: 0.03348862752318382
the nullreferenceexception is thrown when an application attempts to use null in a case where an object is required., confidence: 0.012254065833985806
there is a proprietary issue tracking product developed by atlassian that allows bug tracking and agile project management, confidence: 0.5197254419326782
this is a proprietary issue tracking product developed by atlassian that allows bug tracking and agile project management, confidence: 0.440562278

In [ ]:
# test NL sentnece

examples = {

'yield' : "[MASK] is a keyword that is used like return, except the function will return a generator.",

'itertools' : "The [MASK] module contains special functions to manipulate iterables.",

'--delete' : "The -d option is an alias for [MASK]",

'Flutter' : "[MASK] is an open source framework by Google for building beautiful, natively compiled, multi-platform applications from a single codebase.",

'Git' : "[MASK] is a free and open source distributed version control system designed to handle everything from small to very large projects with speed and efficiency.",

}

for i, example in enumerate(examples):
  for prediction in fill_mask(example):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")
  print("="*50)


In [ ]:
# test PL sentnece

examples = {

'const start' :
"""
const relativeTime = (() => {
    [MASk] = Date.now();
    return () => Date.now() - start;
})();
""",

'i % 3' :
"""
for i in range(1, 100):
    if [MASK] == 0:
        print("multiples of 3", i)
""",

'while' :
"""
int main(void)
{
    int num = 0;
    
    [MASK](num < 5)
    {
        printf("\%d");
        num++;
    }
    return 0;
}
"""
}

for i, example in enumerate(examples):
  for prediction in fill_mask(example):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")
  print("="*50)
